In [ ]:
import os
import pandas as pd
import random
from os.path import join
import shutil
from PIL import Image
import numpy as np

# COUNT CARDINALITIES FOR CLASSES

In [ ]:
LABEL_PATH = os.path.join("data", "forecast_key_dataset.csv")
df = pd.read_csv(LABEL_PATH)

echos = [(df['echogenicity'] == 1).sum(), (df['echogenicity'] == 2).sum(), (df['echogenicity'] == 3).sum(), (df['echogenicity'] == 4).sum(), ]
homos = [(df['homogenicity'] == 1).sum(), (df['homogenicity'] == 2).sum()] 
print(echos)
class_cardinalities = np.array(echos)
n_classes = len(class_cardinalities)
N = class_cardinalities.sum()

weights = N / (n_classes * class_cardinalities)
print(weights)


print(homos)
class_cardinalities = np.array(homos)
n_classes = len(class_cardinalities)
N = class_cardinalities.sum()

weights = N / (n_classes * class_cardinalities)
print(weights)


In [ ]:
INPUT_DATA = os.path.join("data", "segmentation_data", "data", "trans")
INPUT_SEG = os.path.join("data", "segmentation_data", "references", "trans")

LABEL_PATH = os.path.join("parameters_computation", "data", "data_with_attributes.csv")

def count_green_pixels(image_path):
    # Load the image
    green_range = ([0, 100, 0], [150, 255, 150]) 
    image = Image.open(image_path)
    image_np = np.array(image)

    # Define the range for green pixels
    lower_green, upper_green = green_range
    # Count green pixels
    green_pixels = np.all(np.logical_and(lower_green <= image_np, image_np <= upper_green), axis=-1)
    count = np.sum(green_pixels)
    return count



def create_balanced_homogenicity(path_img, path_seg):

    files_img = os.listdir(path_img)
    files_seg_unprocessed = os.listdir(path_seg)
    print(f'files segmentation all: {len(files_seg_unprocessed)}')
    files_seg = []
    for f in files_seg_unprocessed:
        image_path = join(path_seg, f)
        if count_green_pixels(image_path) > 1000:
            files_seg.append(f)
    print(f'files plaque big enopugh all: {len(files_seg)}')

    labels_df = pd.read_csv(LABEL_PATH)
    labels_df.drop_duplicates(subset=['img_name'], keep=False, inplace=True)
    files_labels_class1 = []
    files_labels_class2 = []
    for i, row in labels_df.iterrows():
        if pd.notna(row["echogenicity"]) and pd.notna(row["homogenicity"]):
            if row["homogenicity"] == 1:
                files_labels_class1.append(row["img_name"])
            elif row["homogenicity"] == 2:
                files_labels_class2.append(row["img_name"])
            else:
                print(row["homogenicity"])
    
    class1 = list(set(files_img) & set(files_seg) & set(files_labels_class1))
    class2 = list(set(files_img) & set(files_seg) & set(files_labels_class2))

    # class1 = []
    # class2 = []
    # for item in common_items:


    return class1, class2

input_experiment_path = "balanced_annotated"
c1, c2 = create_balanced_homogenicity(INPUT_DATA,INPUT_SEG)
random.shuffle(c1)
random.shuffle(c2)
min_len = min(len(c1), len(c2))
c1 = c1[:min_len]
c2 = c2[:min_len]
print(len(c1), len(c2))
# image_files = c1 + c2
random.shuffle(c1)
random.shuffle(c2)

train_ratio = 0.8
val_ratio = 0.1
num_train = int(train_ratio * len(c1))
num_val = int((train_ratio+val_ratio) * len(c1))

train_set = c1[:num_train] + c2[:num_train]
val_set = c1[num_train:num_val] + c2[num_train:num_val]
test_set = c1[num_val:] + c2[num_val:]

print(len(train_set), len(val_set), len(test_set))


images_folder = INPUT_DATA
labels_folder = INPUT_SEG

dir_path = os.path.join("parameters_computation", "data", input_experiment_path)

# Create train and validation folders if not exists
os.makedirs(dir_path)

os.makedirs(join(dir_path, "train"))
os.makedirs(join(dir_path, "val"))

train_images_path =join(dir_path, "train", "images")
val_images_path = join(dir_path, "val","images")
train_segmentaion_path = join(dir_path, "train", "segmentations")
val_segmentations_path = join(dir_path, "val", "segmentations")

test_images_path =join(dir_path, "test", "images")
test_segmentations_path =join(dir_path, "test", "segmentations")


os.makedirs(train_images_path)
os.makedirs(val_images_path)
os.makedirs(train_segmentaion_path)
os.makedirs(val_segmentations_path)
os.makedirs(test_images_path)
os.makedirs(test_segmentations_path)

# Copy images to train folder
for image_file in train_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(train_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy labels to train folder
for label_file in train_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(train_segmentaion_path, label_file)
    shutil.copy(src_label, dst_label)

# Copy remaining images to validation folder
for image_file in val_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(val_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy remaining labels to validation folder
for label_file in val_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(val_segmentations_path, label_file)
    shutil.copy(src_label, dst_label)


# Copy remaining images to validation folder
for image_file in test_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(test_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy remaining labels to validation folder
for label_file in test_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(test_segmentations_path, label_file)
    shutil.copy(src_label, dst_label)

train_folder = join(dir_path, "train", "images")
test_folder = join(dir_path, "test", "images")
val_folder = join(dir_path, "val", "images")

train_files = os.listdir(train_folder)
val_files = os.listdir(val_folder)
test_files = os.listdir(test_folder)

print(len(train_files))
print(len(val_files))
print(len(test_files))

df = pd.read_csv(LABEL_PATH)

count1 = len(df[(df['homogenicity'] == 1) & (df['img_name'].isin(train_files))])
count2 = len(df[(df['homogenicity'] == 2) & (df['img_name'].isin(train_files))])
print(f"train {count1}, {count2}")

count1 = len(df[(df['homogenicity'] == 1) & (df['img_name'].isin(test_files))])
count2 = len(df[(df['homogenicity'] == 2) & (df['img_name'].isin(test_files))])
print(f"test_files {count1}, {count2}")

count1 = len(df[(df['homogenicity'] == 1) & (df['img_name'].isin(val_files))])
count2 = len(df[(df['homogenicity'] == 2) & (df['img_name'].isin(val_files))])
print(f"val_files {count1}, {count2}")


In [ ]:
def create_balanced_echogenicity(path_img, path_seg):

    files_img = os.listdir(path_img)
    files_seg = os.listdir(path_seg)

    labels_df = pd.read_csv(os.path.join("parameters_computation", "data", "forecast_key_dataset.csv"))
    labels_df.drop_duplicates(subset=['img_name'], keep=False, inplace=True)
    files_labels_class1 = []
    files_labels_class2 = []
    files_labels_class3 = []
    files_labels_class4 = []
    for i, row in labels_df.iterrows():
        if pd.notna(row["echogenicity"]) and pd.notna(row["homogenicity"]):
            if row["echogenicity"] == 1:
                files_labels_class1.append(row["img_name"])
            elif row["echogenicity"] == 2:
                files_labels_class2.append(row["img_name"])
            elif row["echogenicity"] == 3:
                files_labels_class3.append(row["img_name"])
            elif row["echogenicity"] == 4:
                files_labels_class4.append(row["img_name"])
            else:
                print(row["homogenicity"])
    
    class1 = list(set(files_img) & set(files_seg) & set(files_labels_class1))
    class2 = list(set(files_img) & set(files_seg) & set(files_labels_class2))
    class3 = list(set(files_img) & set(files_seg) & set(files_labels_class3))
    class4 = list(set(files_img) & set(files_seg) & set(files_labels_class4))

    print(len(class1),len(class2),len(class3), len(class4))
    # class1 = []
    # class2 = []
    # for item in common_items:


    return class1, class2, class3, class4

c1, c2, c3, c4 = create_balanced_echogenicity(os.path.join("data","carotid_key_inpainted_dataset","UNET_best_model_segmentation","localized"),os.path.join("data","carotid_key_inpainted_dataset","UNET_best_model_segmentation","segmentations"))
random.shuffle(c1)
random.shuffle(c2)
random.shuffle(c3)
random.shuffle(c4)
min_len = min(len(c1), len(c2), len(c3), len(c4))
c1 = c1[:min_len]
c2 = c2[:min_len]
c3 = c3[:min_len]
c4 = c4[:min_len]

print(len(c1), len(c2), len(c3), len(c4))
# image_files = c1 + c2
random.shuffle(c1)
random.shuffle(c2)
random.shuffle(c3)
random.shuffle(c4)

train_ratio = 0.8
val_ratio = 0.1
num_train = int(train_ratio * len(c1))
num_val = int((train_ratio+val_ratio) * len(c1))

train_set = c1[:num_train] + c2[:num_train] + c3[:num_train] + c4[:num_train]
val_set = c1[num_train:num_val] + c2[num_train:num_val] + c3[num_train:num_val] + c4[num_train:num_val]
test_set = c1[num_val:] + c2[num_val:] + c3[num_val:] + c4[num_val:]

print(len(train_set), len(val_set), len(test_set))


images_folder = os.path.join("data","carotid_key_inpainted_dataset","UNET_best_model_segmentation","localized")
labels_folder = os.path.join("data","carotid_key_inpainted_dataset","UNET_best_model_segmentation","segmentations")
dir_path = os.path.join("parameters_computation", "data", "best_unet_balanced_echogenicity_labels")



# Create train and validation folders if not exists
os.makedirs(dir_path)

os.makedirs(join(dir_path, "train"))
os.makedirs(join(dir_path, "val"))

train_images_path =join(dir_path, "train", "images")
val_images_path = join(dir_path, "val","images")
train_segmentaion_path = join(dir_path, "train", "segmentations")
val_segmentations_path = join(dir_path, "val", "segmentations")

test_images_path =join(dir_path, "test", "images")
test_segmentations_path =join(dir_path, "test", "segmentations")


os.makedirs(train_images_path)
os.makedirs(val_images_path)
os.makedirs(train_segmentaion_path)
os.makedirs(val_segmentations_path)
os.makedirs(test_images_path)
os.makedirs(test_segmentations_path)

# Copy images to train folder
for image_file in train_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(train_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy labels to train folder
for label_file in train_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(train_segmentaion_path, label_file)
    shutil.copy(src_label, dst_label)

# Copy remaining images to validation folder
for image_file in val_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(val_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy remaining labels to validation folder
for label_file in val_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(val_segmentations_path, label_file)
    shutil.copy(src_label, dst_label)


# Copy remaining images to validation folder
for image_file in test_set:
    src_image = os.path.join(images_folder, image_file)
    dst_image = os.path.join(test_images_path, image_file)
    shutil.copy(src_image, dst_image)

# Copy remaining labels to validation folder
for label_file in test_set:
    src_label = os.path.join(labels_folder, label_file)
    dst_label = os.path.join(test_segmentations_path, label_file)
    shutil.copy(src_label, dst_label)

train_folder = join("parameters_computation", "data", "best_unet_balanced_echogenecity_labels", "train", "images")
test_folder = join("parameters_computation", "data", "best_unet_balanced_echogenecity_labels", "test", "images")
val_folder = join("parameters_computation", "data", "best_unet_balanced_echogenecity_labels", "val", "images")

train_files = os.listdir(train_folder)
val_files = os.listdir(val_folder)
test_files = os.listdir(test_folder)

print(len(train_files))
print(len(val_files))
print(len(test_files))

df = pd.read_csv(os.path.join("parameters_computation", "data", "forecast_key_dataset.csv"))

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(train_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(train_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(train_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(train_files))])
print(f"train {count1}, {count2}, {count3}, {count4}")

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(test_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(test_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(test_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(test_files))])
print(f"test_files {count1}, {count2}, {count3}, {count4}")

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(val_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(val_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(val_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(val_files))])
print(f"val_files {count1}, {count2}, {count3}, {count4}")


In [ ]:
train_folder = join("parameters_computation", "data", "best_unet_balanced_echogenicity_labels", "train", "images")
test_folder = join("parameters_computation", "data", "best_unet_balanced_echogenicity_labels", "test", "images")
val_folder = join("parameters_computation", "data", "best_unet_balanced_echogenicity_labels", "val", "images")

train_files = os.listdir(train_folder)
val_files = os.listdir(val_folder)
test_files = os.listdir(test_folder)

print(len(train_files))
print(len(val_files))
print(len(test_files))

df = pd.read_csv(os.path.join("parameters_computation", "data", "forecast_key_dataset.csv"))

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(train_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(train_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(train_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(train_files))])
print(f"train {count1}, {count2}, {count3}, {count4}")

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(test_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(test_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(test_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(test_files))])
print(f"test_files {count1}, {count2}, {count3}, {count4}")

count1 = len(df[(df['echogenicity'] == 1) & (df['img_name'].isin(val_files))])
count2 = len(df[(df['echogenicity'] == 2) & (df['img_name'].isin(val_files))])
count3 = len(df[(df['echogenicity'] == 3) & (df['img_name'].isin(val_files))])
count4 = len(df[(df['echogenicity'] == 4) & (df['img_name'].isin(val_files))])
print(f"val_files {count1}, {count2}, {count3}, {count4}")


In [ ]:
from PIL import Image
import numpy as np
from os import listdir
from os.path import join
def count_green_pixels(image_path, green_range):
    # Load the image
    image = Image.open(image_path)
    image_np = np.array(image)

    # Define the range for green pixels
    lower_green, upper_green = green_range

    # Count green pixels
    green_pixels = np.all(np.logical_and(lower_green <= image_np, image_np <= upper_green), axis=-1)
    count = np.sum(green_pixels)

    return count

# Example usage
green_range = ([0, 100, 0], [150, 255, 150])  # Adjust these values based on your definition of green

count = 0
count2 = 0
seg_path = "data//carotid_key_inpainted_dataset//localization9_UNET_best_model_segmentation_trainvaltest//segmentations"
files = listdir(seg_path)
print(len(files))
for f in files:
    image_path = join(seg_path, f)
    if count_green_pixels(image_path, green_range) > 1000:
        count +=1
    else:
        count2 += 1

print(count)
print(count2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'images' is a list of 3D tensors (channel, width, height)
# Example: images = [np.random.rand(3, 100, 100), np.random.rand(3, 100, 100)]

def non_zero_pixels(image):
    # Filter out pixels where any channel is zero
    mask = np.all(image > 0, axis=0)
    return image[:, mask]

def mean_intensity_values(filtered_image):
    # Calculate mean intensity for each channel
    return np.mean(filtered_image, axis=(1, 2))

# Collect mean intensity values for each channel
channel_means = {0: [], 1: [], 2: []}

for image in images:
    filtered_image = non_zero_pixels(image)
    means = mean_intensity_values(filtered_image)
    for i, mean in enumerate(means):
        channel_means[i].append(mean)

# Create boxplots
plt.boxplot([channel_means[0], channel_means[1], channel_means[2]], labels=['Channel 1', 'Channel 2', 'Channel 3'])
plt.title('Boxplot of Mean Intensity Values for Each Channel')
plt.ylabel('Mean Intensity')
plt.show()

In [ ]:
df = 